In [95]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-machine-learning-from-disaster/train.csv
/kaggle/input/titanic-machine-learning-from-disaster/test.csv


In [96]:
data = pd.read_csv("/kaggle/input/titanic-machine-learning-from-disaster/train.csv")
test = pd.read_csv("/kaggle/input/titanic-machine-learning-from-disaster/test.csv")
test_ids = test["PassengerId"]

In [97]:
data.shape

(891, 12)

In [98]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [99]:
data.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [100]:
a = data.columns
for col_name in a:
    print('Null values in {} column = {}'.format(col_name, data[col_name].isnull().sum()))

Null values in PassengerId column = 0
Null values in Survived column = 0
Null values in Pclass column = 0
Null values in Name column = 0
Null values in Sex column = 0
Null values in Age column = 177
Null values in SibSp column = 0
Null values in Parch column = 0
Null values in Ticket column = 0
Null values in Fare column = 0
Null values in Cabin column = 687
Null values in Embarked column = 2


In [101]:
def clean(data):
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"],axis=1)
    
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
    data.Embarked.fillna("U", inplace=True)
    return data

data = clean(data)
test = clean(test)

In [102]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [103]:
#dict_gender = {'male':0, 'female':1}
#data['Sex'] = data['Sex'].replace(['female','male'],[0,1])

#OR
#data['Sex'] = data['Sex'].map(dict_gender)
#data.head(5)

In [104]:
for col in data:
    print('{} column has {} unique values'.format(col,data[col].unique().shape[0]))

Survived column has 2 unique values
Pclass column has 3 unique values
Sex column has 2 unique values
Age column has 88 unique values
SibSp column has 7 unique values
Parch column has 7 unique values
Fare column has 248 unique values
Embarked column has 4 unique values


In [105]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
U      2
Name: Embarked, dtype: int64

In [106]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)

data.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


DDDdd

**Using Sklearn Logistic Regression**

In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = data["Survived"]
X = data.drop(["Survived"],axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.02, random_state=42)

In [108]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [109]:
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score

accuracy_score(y_val, predictions)

0.8888888888888888

In [110]:
submission_preds = clf.predict(test)

In [111]:
df = pd.DataFrame({"PassengerId": test_ids.values,
                   "Survived":submission_preds
                  })

In [112]:
df.to_csv("submission.csv", index=False)

In [114]:
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv